First of all, I would like to show my appreciation to the owners' of the follwoing Notebooks that were extremly helpful:
https://www.kaggle.com/allunia/santander-customer-transaction-eda
https://www.kaggle.com/gpreda/santander-eda-and-prediction
https://www.kaggle.com/jiweiliu/lgb-2-leaves-augment

# Part 1 - EDA

In [ ]:
import pandas as pd
import numpy as np
#import lightgbm as lgb
from sklearn.metrics import roc_auc_score
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore')
plt.style.use('seaborn')
random_state = 42
np.random.seed(random_state)
from sklearn.model_selection import cross_validate
# visualization
sns.set()

In [ ]:
def data_load(filename):
    return pd.read_csv(filename)

def basic_EDA(df):
    size = df.shape
    sum_duplicates = df.duplicated().sum()
    sum_null = df.isnull().sum().sum()
    return print("Number of Samples: %d,\nNumber of Features: %d,\nDuplicated Entries: %d,\nNull Entries: %d" %(size[0],size[1], sum_duplicates, sum_null))

#Plot Bar graph with all classes and percentages - Return number of Classes and Samples per class
def bar_plot(df, target):
    unique, counts = np.unique(target, return_counts = True)
    label = np.zeros(len(unique))
    for i in range(len(unique)):
        label[i] = (counts[i]/df.shape[0])*100
        plt.bar(unique,counts, color = ['burlywood', 'green'], edgecolor='black')
        plt.text(x = unique[i]-0.15, y = counts[i]+0.01*df.shape[0], s = str("%.2f%%" % label[i]), size = 15)
    plt.ylim(0, df.shape[0])
    plt.xticks(unique)
    plt.xlabel("Target")
    plt.ylabel("Count")
    plt.show()
    return unique, counts

#Plots Heatmap and top 10 and bottom correlated features
def feat_corr_analysis(corrmat):
    f, ax = plt.subplots(figsize =(9, 8)) 
    #1 Heatmap
    sns.heatmap(corrmat, vmin=0, vmax=0.2, ax = ax, cmap ="YlGnBu", linewidths = 0.1)
    plt.title("Heatmap - Correlation between data variables")
    #2 Correlation Values and Features
    correlations = corrmat.abs().unstack().sort_values(kind="quicksort").reset_index()
    correlations = correlations[correlations['level_0'] != correlations['level_1']]
    #Top 10 correlated features
    correlations.tail(10)
    #Bottom 10 correlated features
    correlations.head(10)
    return correlations.tail(10)

def feat_corr_distr(train,test):
    #Plot distribution of Feature Correlation
    train_corr_distr = train.values.flatten()
    train_corr_distr = train_corr_distr[train_corr_distr != 1]
    test_corr_distr = test.values.flatten()
    test_corr_distr = test_corr_distr[test_corr_distr != 1]
    plt.figure(figsize=(20,5))
    sns.distplot(train_corr_distr, color="Red", label="Train")
    sns.distplot(test_corr_distr, color="black", label="Test")
    plt.xlabel("Correlation values")
    plt.ylabel("Density")
    plt.title("Feature Correlation"); 
    plt.legend();
    
def prediction(x_train,y_train):
    classifier.fit(x_train,y_train)
    y_proba = classifier.predict_proba(x_train)
    plt.figure(figsize=(20,5))
    y_pred = classifier.predict(x_train)
    y_proba = classifier.predict_proba(x_train)
    score = roc_auc_score(y_train, y_pred)
    return y_proba, score

def probability_class(y_proba):
    plt.figure(figsize=(20,5))
    sns.distplot(y_proba[y_train==1,1], label="True Class 1")
    sns.distplot(y_proba[y_train==0,1], label="True Class 0")
    plt.xticks(np.arange(0,1, 0.1))
    plt.xlabel("Predicted probability values of class 1")
    plt.ylabel("Density")
    plt.title("Predicted probability values of class 1 against the true Target"); 
    plt.legend();

Importing the Training and Test set CSV files.

In [ ]:
#Load Data
train = data_load('../input/santander-customer-transaction-prediction/train.csv')
test = data_load('../input/santander-customer-transaction-prediction/test.csv')

## Overview
For an overview of the data, the function outputs the number of samples, features, duplicated and null values:

In [ ]:
print("***Train EDA***")
train_EDA = basic_EDA(train)
print("***Test EDA***")
test_EDA  = basic_EDA(test)


This is a large dataset, with 200.000 samples and over 200 features. No null values or duplicated entries were found, eliminating the need for data cleanse at this stage. It is also important to understand the type of variables present in this dataset:

In [ ]:
train.info()
train.head(10)

The "target" column contains the true labels. The "ID_CODE" is the object dtype mentioned in the info(). Other than this, all values are numeric so there is no need for enconding. This is true for train and test set. 

In [ ]:
test.info()
test.head(10)

At this point, it is possible to remove the ID_code column and replace it with a numerical index that can be easily handled by the ML models. The new column is called Id.

In [ ]:
#ID_Code is the only object dtype. It can be replaced by index values
train["Id"] = train.index.values
test["Id"] = test.index.values
init_train_ID = train.ID_code.values
init_test_ID = test.ID_code.values
train.drop("ID_code", axis=1, inplace=True)
test.drop("ID_code", axis=1, inplace=True)
train.head(5)

## Samples per Class
By analysing the samples per class ratio it is possible to see the class imbalance issue. Almost 90% of the samples are clients that did not performed the transaction. An initial attempt was made to use SMOTE and ADASYN as oversampling methods. Preliminary test have shown that this did not helped the model to generalise to the test set.

In [ ]:
##Visualise Class Imbalance - Training Set
num_classes, feat_per_class = bar_plot(train, train["target"])

## Feature Correlation
Since this dataset has a significant amount of features, correlated helps to develop an understanding on how the variable relate to each other. This can indicate starting points for feature engineering and the most relevant variables for the model. 

In [ ]:
train_corrmat = train.drop(["target"], axis=1).corr()
feat_corr_train = feat_corr_analysis(train_corrmat)
test_corrmat = test.corr()
feat_corr_test = feat_corr_analysis(test_corrmat)


There is none or very little correlation between the features for training and test sets. The plot below shows that the correlation values distribution is similar for the train and test set.

In [ ]:
feat_corr_distr(train_corrmat, test_corrmat)

# Part 2 - Model Baseline
Random Forest (RF) algorithm is used as a starting point to allow better understanding of this dataset. RF is quick, it does not have that many parameters to tune and usually provide reasonable results. By understanding what works and what does not works with RF, it is possible to improve the model quicker and then try other algorithms. This first step is mainly to understand the top important features, apply feature engineering and sampling techniques.

In [ ]:
#Prepare DF
X_train = train.drop("target", axis=1).values
y_train = train.target.values

In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(test)

Below is the RF classifier. Grid Search supported the selection of the initial parameters:

In [ ]:
#Random Forest Baseline Model
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier(n_estimators = 10, 
                       criterion = 'gini',
                       max_depth = 15, 
                       max_features = 'auto', 
                       min_samples_leaf = 1, 
                       random_state = 0)

The code below creates the RF model and outputs the AUC result for the training set. It also outputs the probabilities of each sample belonging to class 0 or 1.

In [ ]:
RF_y_proba, RF_model_score = prediction(X_train,y_train)
print("Baseline RF: %.2f "%(RF_model_score))
pd.DataFrame(RF_y_proba).describe()


RF_y_proba contains the probabilites of class 0 and 1 for every sample. Most values for class 1 are small and the majority does not reach 0.1, as shown by the table above. One hypthesis can be that the usual threshold of 0.5 is not ideal for this model to assign class 1 or 0. One way to visualise this is to plot the "Class 1" probability distribution for all samples, according to their True label. 

In [ ]:
probability_class(RF_y_proba)

From the plot above it is possible to extract the following insights:
* Most of the True Class 1 samples were predicted by our model with a probability lower than 0.5
* Most of the True Class 0 samples are concentrated within the values of 0 and 0.15
* For samples with Class 1 probability values higher than approximately 0.15, it is almost safe to say they are True Class 1. Altough there is the orange bump around 0.17
* It is possible to perform a quick test and see if this helps the model accuracy:

In [ ]:
#From the graph, used a threshold of 0.15
threshold = 0.15
y_pred = np.zeros(RF_y_proba.shape[0])
y_pred[RF_y_proba[:,1] >= threshold] = 1
RFT_model_score = roc_auc_score(y_train, y_pred)
print("Baseline RF: %.2f \nThreshold RF: %.2f"%(RF_model_score, RFT_model_score))

A 30% increase in accuracy was achieved by this simple but effective strategy. This is something to keep in mind in the future and check if other models also present this behaviour.

## Feature Importances
This analysis can helps reduce the number of features while mantaining the minimum impact on model accuracy. Using the RF classifier, the following features are more effective to our model:


In [ ]:
importances = classifier.feature_importances_
indices = np.argsort(importances)[::-1]
colors = plt.cm.Reds(importances)
# Plot the feature importances
plt.figure(1, figsize=(40,20))
plt.title("Feature importances")
sns.barplot(x=indices, y=importances[indices], order = indices,palette="Blues_d")
plt.xticks(rotation = 90)
plt.show()

In [ ]:
#From the Feature Importances, we can see that 
n_top = 50
idx = np.argsort(importances)[::-1][0:n_top]
feature_names = train.drop("target", axis=1).columns.values
plt.figure(figsize=(20,5))
sns.barplot(x=feature_names[idx], y=importances[idx],palette="Blues_d");
plt.title("Top important features to start");
plt.xticks(rotation = 90)
plt.show()

In [ ]:
pd.DataFrame(X_train[:,idx]).describe()

The next model is built using the top 50 features. Additional features were created performing the following operations per row: 

In [ ]:
X_train_top = X_train[:,0:n_top]
sum_feat= X_train_top.sum(axis=1)
min_feat = X_train_top.min(axis=1)
max_feat = X_train_top.max(axis=1)
mean_feat = X_train_top.mean(axis=1)
std_feat = X_train_top.std(axis=1)
X_train_top = np.concatenate((X_train_top,
                             sum_feat[:,None],
                             min_feat[:,None],
                             max_feat[:,None],
                             mean_feat[:,None],
                             std_feat[:,None]),
                             axis = 1)
RF1_y_proba, RF1_model_score = prediction(X_train_top, y_train)
probability_class(RF1_y_proba)

With the new features, is necessary to revaluate and assign a new threshold 

In [ ]:
RF1_y_pred = np.zeros(RF1_y_proba.shape[0])
RF1_y_pred[RF1_y_proba[:,1] >= 0.12] = 1
RF1_model_score = roc_auc_score(y_train, y_pred)
print("Baseline RF %.2f\nThreshold RF %.2f\nThreshold RF with only top features %.2f"%(RF_model_score, RFT_model_score, RF1_model_score))

# Part 3 - Submission
By reducing the number of features to a quarter of the original model, the same accuracy was mantained. At this point is interesting to check our score in the test set

In [ ]:
X_test_top = X_test[:,idx]
X_test_top = X_test[:,0:n_top]
sum_feat= X_test_top.sum(axis=1)
min_feat = X_test_top.min(axis=1)
max_feat = X_test_top.max(axis=1)
mean_feat = X_test_top.mean(axis=1)
std_feat = X_test_top.std(axis=1)
X_test_top = np.concatenate((X_test_top,
                             sum_feat[:,None],
                             min_feat[:,None],
                             max_feat[:,None],
                             mean_feat[:,None],
                             std_feat[:,None]),
                             axis = 1)
#####################################################
y_proba = classifier.predict_proba(X_test_top)
y_pred = np.zeros(y_proba.shape[0])
y_pred[y_proba[:,1] >= 0.12] = 1
submission = pd.concat([pd.DataFrame(init_test_ID),pd.DataFrame(y_pred)],axis = 1)
submission.columns = ['ID_code', 'Target']
submission.to_csv("submission_RForest.csv", index=False)

This concludes a quick analysis on the Santander Customer Transaction Challenge. The result on the test set points to overfitting, since it only achieved 0.59.
On the following days, I want to devote more attention to other models and improve feature engineering. I noticed many challenge submissions using Light GBM and ANN. It is also essential to implement an ensemble, as this usually improves the overall model score.